In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

import os, pickle, re, glob, time
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
from collections import Counter

sns.set_style('ticks')
pd.set_option('precision', 2)
#np.set_printoptions('precision', 2)

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# โหลด distance matrix + clustering

In [2]:
args = pickle.load(open('arguments.pickle', 'rb'))

In [3]:
dm = pickle.load(open(os.path.join(args.preprocess_dir, "dist_mat_X_df.pickle"), "rb"))
dm

array([[0.  , 0.  , 0.02, ..., 0.7 , 0.17, 0.72],
       [0.  , 0.  , 0.02, ..., 0.7 , 0.17, 0.72],
       [0.02, 0.02, 0.  , ..., 0.72, 0.19, 0.74],
       ...,
       [0.7 , 0.7 , 0.72, ..., 0.  , 0.69, 0.07],
       [0.17, 0.17, 0.19, ..., 0.69, 0.  , 0.7 ],
       [0.72, 0.72, 0.74, ..., 0.07, 0.7 , 0.  ]])

In [4]:
dm.shape

(3406, 3406)

In [5]:
clade_df = pd.read_csv(os.path.join(args.preprocess_dir, 'clade_df.csv')).sort_values('Accession ID')
clade_df.head()

,Accession ID,Clade,Lineage,Collection date,Year,Month,Year-Month,WHO name
2545,406973,L,B.1.36.10,2020-01-23,2020,1,2020-01,Others
2546,407987,L,B,2020-01-25,2020,1,2020-01,Others
2547,407988,O,B,2020-02-01,2020,2,2020-02,Others
351,410535,S,A,2020-02-03,2020,2,2020-02,Others
352,410536,O,B,2020-02-06,2020,2,2020-02,Others


In [6]:
clade_df['Clade'] = clade_df['Clade'].replace(np.nan,'O')

In [7]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=clade_df['WHO name'].unique().shape[0], 
                random_state=0, precompute_distances=False).fit(dm)

In [8]:
clade_df.loc[:, 'k_mean'] = [str(i) for i in kmeans.labels_]

In [9]:
clade_df.to_csv(os.path.join(args.preprocess_dir, 'clade_kmean.csv'), index=False)

In [10]:
X_id_list = pickle.load(open(os.path.join(args.preprocess_dir, "X_id_list.pickle"), "rb"))
rand_pos = pickle.load(open(os.path.join(args.preprocess_dir, "rand_pos.pickle"), "rb"))
rand_pos = sorted(rand_pos)

In [11]:
id_sampling = X_id_list[rand_pos]

In [12]:
clade_df = clade_df.set_index('Accession ID').loc[id_sampling].reset_index()
clade_df.columns = ['Accession ID'] + list(clade_df.columns[1:])
clade_df.head()

,Accession ID,Clade,Lineage,Collection date,Year,Month,Year-Month,WHO name,k_mean
0,406973,L,B.1.36.10,2020-01-23,2020,1,2020-01,Others,5
1,407987,L,B,2020-01-25,2020,1,2020-01,Others,5
2,407988,O,B,2020-02-01,2020,2,2020-02,Others,5
3,410535,S,A,2020-02-03,2020,2,2020-02,Others,5
4,410536,O,B,2020-02-06,2020,2,2020-02,Others,5


## TSE

In [13]:
# หา TSE เฉพาะ sampling data
dm_sampling = dm[rand_pos][:,rand_pos]
dm_sampling

array([[0.  , 0.  , 0.02, ..., 0.7 , 0.17, 0.72],
       [0.  , 0.  , 0.02, ..., 0.7 , 0.17, 0.72],
       [0.02, 0.02, 0.  , ..., 0.72, 0.19, 0.74],
       ...,
       [0.7 , 0.7 , 0.72, ..., 0.  , 0.69, 0.07],
       [0.17, 0.17, 0.19, ..., 0.69, 0.  , 0.7 ],
       [0.72, 0.72, 0.74, ..., 0.07, 0.7 , 0.  ]])

In [14]:
dm_sampling.shape

(3406, 3406)

In [15]:
from sklearn.manifold import TSNE
model = TSNE(metric="precomputed", n_components=2, perplexity=10.0, learning_rate=5.0, random_state=0)  #10,1
Y = model.fit_transform(dm_sampling) 

In [16]:
clade_df.head()

,Accession ID,Clade,Lineage,Collection date,Year,Month,Year-Month,WHO name,k_mean
0,406973,L,B.1.36.10,2020-01-23,2020,1,2020-01,Others,5
1,407987,L,B,2020-01-25,2020,1,2020-01,Others,5
2,407988,O,B,2020-02-01,2020,2,2020-02,Others,5
3,410535,S,A,2020-02-03,2020,2,2020-02,Others,5
4,410536,O,B,2020-02-06,2020,2,2020-02,Others,5


In [17]:
Y_df = pd.DataFrame(Y,columns=['x','y'])
Y_df['k_mean'] = clade_df['k_mean']
Y_df['result'] = clade_df['Clade']
Y_df['lineage'] = clade_df['Lineage']
Y_df['who_name'] = clade_df['WHO name']

In [18]:
Y_df.head()

,x,y,k_mean,result,lineage,who_name
0,17.45,-1.98,5,L,B.1.36.10,Others
1,17.47,-1.97,5,L,B,Others
2,14.91,0.08,5,O,B,Others
3,-4.07,-28.73,5,S,A,Others
4,24.32,-5.82,5,O,B,Others


In [19]:
pickle.dump(Y_df, open(os.path.join(args.preprocess_dir, 'Y_tse.pickle'), 'wb'))

In [23]:
# from lib import plot_data
# fig_k_mean = plot_data.fig_cluster(os.path.join(args.preprocess_dir, 'Y_tse.pickle'), 'k_mean')

# # fig_k_mean.write_image("../pic/fig_k_mean.png", scale=3)
# fig_k_mean.show()